In [1]:
# tutorial inspiration from:
# https://www.kaggle.com/mohitr/topic-modeling-of-paper-abstracts

#!pip install sklearn
#!pip install gensim
#!pip install pyLDAvis

import pandas as pd


In [2]:
# Load in the dataframe
df = pd.read_csv("/home/mrinalmanu/Desktop/metagen_project/BIG_DF")

In [3]:
df.head(1)

,rank display,country code,country,size,focus,research,age band,doi,year,number_authors,...,title,abstract,author,affiliation_y,latitude,longitude,continent,fuzzy_matched_QS,original_affiliation,Token_Set_Ratio
0,1,US,United States,M,CO,VH,5.0,10.2196/22635,2020,14,...,Natural Language Processing Reveals Vulnerable...,The COVID-19 pandemic is impacting mental heal...,Laurie Rumker,Department of Brain and Cognitive Sciences Mas...,37.09024,-95.712891,North America,Massachusetts Institute of Technology (MIT),Department of Brain and Cognitive Sciences Mas...,95


In [4]:
list(df.columns.values) 

['rank display',
 'country code',
 'country',
 'size',
 'focus',
 'research',
 'age band',
 'doi',
 'year',
 'number_authors',
 'journal',
 'title',
 'abstract',
 'author',
 'affiliation_y',
 'latitude',
 'longitude',
 'continent',
 'fuzzy_matched_QS',
 'original_affiliation',
 'Token_Set_Ratio']

In [5]:
import pandas as pd 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import gensim
import gensim.corpora as corpora
import re
import numpy as np
from pprint import pprint
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/home/mrinalmanu/miniconda3/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/home/mrinalmanu/miniconda3/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/home/mrinalmanu/miniconda3/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/home/mrinalmanu/miniconda3/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.ab

In [6]:
papers = df.loc[:, ['doi', 'year', 'title', 'abstract', 'author', 'affiliation_y', 'country', 'fuzzy_matched_QS']]

In [7]:
papers.head(n=1)

,doi,year,title,abstract,author,affiliation_y,country,fuzzy_matched_QS
0,10.2196/22635,2020,Natural Language Processing Reveals Vulnerable...,The COVID-19 pandemic is impacting mental heal...,Laurie Rumker,Department of Brain and Cognitive Sciences Mas...,United States,Massachusetts Institute of Technology (MIT)


In [8]:
abstracts = papers.groupby(['abstract'], as_index=False, sort=False).agg(' | '.join)


In [9]:
abstracts

,abstract,title,author,affiliation_y,country,fuzzy_matched_QS
0,The COVID-19 pandemic is impacting mental heal...,Natural Language Processing Reveals Vulnerable...,Laurie Rumker | Tanya Talkar | Daniel M ...,Department of Brain and Cognitive Sciences Mas...,United States | United States | United States ...,Massachusetts Institute of Technology (MIT) | ...
1,"Brain regions in the ""pain matrix"", can be act...",Empathic control through coordinated interacti...,Nir Jacoby | Rebecca Saxe | Emile G Brun...,Department of Brain and Cognitive Sciences Mas...,United States | United States | United States ...,Massachusetts Institute of Technology (MIT) | ...
2,During embryonic development of bilaterally sy...,Characterizing the diverse cells that associat...,N PenkoffLidbeck | C Schneider | M S Sch...,Computational and Systems Biology Massachusett...,United States | United States | Canada | Unite...,Massachusetts Institute of Technology (MIT) | ...
3,Zebra finches (Taeniopygia guttata) are labora...,"Campylobacter taeniopygiae sp. nov., Campyloba...",Erin Bryant | Zeli Shen | Anthony Mannio...,Division of Comparative Medicine Massachusetts...,United States | United States | United States ...,Massachusetts Institute of Technology (MIT) | ...
4,Non-recurrent deep convolutional neural networ...,Evidence that recurrent circuits are critical ...,Kohitij Kar | Kailyn Schmidt | James J D...,McGovern Institute for Brain Research and Depa...,United States | United States | United States ...,Massachusetts Institute of Technology (MIT) | ...
...,...,...,...,...,...,...
2800,This study aims to investigate the effects of ...,Effects of ethical leadership on bullying and ...,Talat Islam | Ishfaq Ahmed,Institute of Business Administration Universit...,Pakistan | Pakistan,University of the Punjab | University of the P...
2801,"Distinctions in illness behavior for women, su...",The combined influence of psychosocial factors...,Jenifer J Thomas | Evelinn A Borrayo,a Fay W Whitney School of Nursing University o...,United States | United States,University of Wyoming | University of Wyoming
2802,This study investigated continuity and stabili...,Taking stress response out of the box: stabili...,Heidemarie K Laurent | Jennifer C Ablow | ...,Department of Psychology University of Wyoming...,United States | United States | United States,University of Wyoming | University of Wyoming ...
2803,The rising prevalence of (online) pornography ...,Adolescent sexual aggressiveness and pornograp...,Azra Tafro | Azra Tafro | Aleksandar ..t...,Department of Electronics Systems and Informat...,Croatia | Croatia | Croatia | Croatia,University of Zagreb | University of Zagreb | ...


In [10]:
# total 2805 papers are there
abstracts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2805 entries, 0 to 2804
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   abstract          2805 non-null   object
 1   title             2805 non-null   object
 2   author            2805 non-null   object
 3   affiliation_y     2805 non-null   object
 4   country           2805 non-null   object
 5   fuzzy_matched_QS  2805 non-null   object
dtypes: object(6)
memory usage: 153.4+ KB


In [11]:
#Removing symbols from Abstracts

abstracts['Abstract_Cleaned'] = abstracts.apply(lambda row: (re.sub("[^A-Za-z0-9' ]+", ' ', row['abstract'])),axis=1)

In [12]:
# tokenization


abstracts['Abstract_Cleaned'] = abstracts.apply(lambda row: (word_tokenize(row['Abstract_Cleaned'])), axis = 1)

In [13]:
#nltk.download('stopwords')
#nltk.download('wordnet')

In [14]:
# we have to remove stopwords

stop_words = set(stopwords.words('english'))
abstracts['Abstract_Cleaned'] = abstracts.apply(lambda row: ([w for w in row['Abstract_Cleaned'] if w not in stop_words]),axis=1)

In [15]:
# creating lemmas

# Lemmatization

lmtzr = WordNetLemmatizer()
abstracts['Abstract_Cleaned'] = abstracts.apply(lambda row: ([lmtzr.lemmatize(w) for w in row['Abstract_Cleaned']]), axis=1)

In [16]:
abstracts['Abstract_Cleaned'][1][:10]

['Brain',
 'region',
 'pain',
 'matrix',
 'activated',
 'observing',
 'reading',
 'others',
 'physical',
 'pain']

# topic model

In [17]:
# Creating Dictionary and Corpus

dictionary = corpora.Dictionary(abstracts['Abstract_Cleaned'])
texts = abstracts['Abstract_Cleaned']
corpus = [dictionary.doc2bow(text) for text in abstracts['Abstract_Cleaned']]

In [18]:
# Building LDA Model

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [19]:
# Printing the Keywords in topics

#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [20]:
# Visualizing topics using pyLDAvis

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.224570 -0.114100       1        1  20.555489
2      0.142174  0.106176       2        1  19.754207
0      0.216396 -0.103692       3        1  18.203351
8      0.104093 -0.099733       4        1  12.007917
5      0.137620  0.072983       5        1  11.059199
7     -0.029653  0.203221       6        1   6.932400
3     -0.117806  0.222998       7        1   5.287462
1     -0.182451 -0.066868       8        1   3.402793
4     -0.234543 -0.160265       9        1   1.918143
6     -0.260401 -0.060720      10        1   0.879038, topic_info=            Term         Freq        Total Category  logprob  loglift
1              0  2371.000000  2371.000000  Default  30.0000  30.0000
776          use  1792.000000  1792.000000  Default  29.0000  29.0000
94        health  2066.000000  2066.000000  Default  28.0000  28.0000
357        model  3145.000000  3145.000000  Default  27.0000  27.0000
160         risk  1131.000000  1131.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
3794   interplay    18.602288    19.556868  Topic10  -5.3132   4.6841
3303     instead    18.312052    19.266597  Topic10  -5.3290   4.6833
3225        seem    18.141729    19.096249  Topic10  -5.3383   4.6828
15289        HRV    18.123717    19.078212  Topic10  -5.3393   4.6828
221        brain    21.879779   277.138872  Topic10  -5.1510   2.1951

[549 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
202       1  0.110973          '
202       2  0.014954          '
202       3  0.873617          '
1         4  0.999593          0
2         4  0.996474        001
...     ...       ...        ...
8005      3  0.996066  workplace
731       1  0.592708       year
731       3  0.009717       year
731       4  0.395139       year
731       9  0.001080       year

[899 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 3, 1, 9, 6, 8, 4, 2, 5, 7])

In [21]:
copy = abstracts

In [22]:
# inverse document frequencies for topics

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

abstracts = copy['abstract'].values

count_vectorizer = CountVectorizer()
counts = count_vectorizer.fit_transform(abstracts)
tfidf_vectorizer = TfidfTransformer().fit(counts)
tfidf_abstracts = tfidf_vectorizer.transform(counts)

In [23]:
# Mapping Top Topics & Abstracts

from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=9, max_iter=10, learning_method='online', learning_offset=50.,random_state=0).fit(tfidf_abstracts)
lda_W = lda_model.transform(tfidf_abstracts)
lda_H = lda_model.components_
#Not sure how to find lda_W & lda_H using gensim lda model

In [24]:
tfidf_abstracts.shape

(2805, 22726)

In [25]:
# Affinity propogation on abstracts
#Testing affinity propogation on abstracts tfidf(experimental)

from sklearn.cluster import AffinityPropagation

X = tfidf_abstracts
clustering = AffinityPropagation().fit(X)
clustering 

/home/mrinalmanu/miniconda3/lib/python3.7/site-packages/sklearn/cluster/_affinity_propagation.py:154: FutureWarning: 'random_state' has been introduced in 0.23. It will be set to None starting from 1.0 (renaming of 0.25) which means that results will differ at every function call. Set 'random_state' to None to silence this warning, or to 0 to keep the behavior of versions <0.23.
  FutureWarning


AffinityPropagation()

In [26]:
abstract_affinity_clusters = list(clustering.labels_)
len(set(abstract_affinity_clusters))


374

In [27]:
# Building LDA Model

lda_model_17 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=17, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.247592 -0.120410       1        1  21.068921
5      0.283235  0.037665       2        1  18.146922
9      0.281352  0.153535       3        1  12.994200
8      0.215116 -0.068668       4        1  11.468040
0      0.220909  0.043555       5        1   8.702299
15     0.130959 -0.210080       6        1   8.524268
2      0.105462  0.176060       7        1   5.777185
7     -0.010434 -0.081849       8        1   3.053508
3     -0.049025  0.224955       9        1   2.402619
11    -0.042331 -0.174544      10        1   2.353583
14    -0.120869 -0.035812      11        1   2.269307
13    -0.192604  0.004732      12        1   0.778000
6     -0.217420  0.005836      13        1   0.737701
1     -0.212119  0.006085      14        1   0.629513
16    -0.208436  0.016128      15        1   0.500979
4     -0.199295  0.019113      16        1   0.500330
12    -0.232092  0.003702      17        1   0.092624, topic_info=            Term         Freq        Total Category  logprob  loglift
162         self  2997.000000  2997.000000  Default  30.0000  30.0000
523     behavior  2667.000000  2667.000000  Default  29.0000  29.0000
776          use  2283.000000  2283.000000  Default  28.0000  28.0000
1              0  2731.000000  2731.000000  Default  27.0000  27.0000
1227      sexual  1066.000000  1066.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
4311  impairment     0.014491     1.078240  Topic17 -10.2205   2.6748
4194        path     0.014495     1.080015  Topic17 -10.2202   2.6734
9281           F     0.014491     1.078223  Topic17 -10.2205   2.6748
6236   inhibitor     0.014493     1.079441  Topic17 -10.2204   2.6738
4170          01     0.014491     1.079338  Topic17 -10.2204   2.6738

[836 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
202       2  0.698493        '
202       5  0.301139        '
0         1  0.134826       's
0         2  0.499979       's
0         3  0.072328       's
...     ...       ...      ...
1713      2  0.945851  working
1713     14  0.052459  working
731       3  0.031020     year
731       4  0.325264     year
731       5  0.642551     year

[1249 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 6, 10, 9, 1, 16, 3, 8, 4, 12, 15, 14, 7, 2, 17, 5, 13])

In [29]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=9, max_iter=10, learning_method='online', learning_offset=50.,random_state=0).fit(tfidf_abstracts)
lda_W = lda_model.transform(tfidf_abstracts)
lda_H = lda_model.components_

In [30]:
# first remove duplicated title, which was due to grouping the dataframe

c_c = copy['title'].str.split("|", 1, expand=True)
copy['title'] = c_c[0]
copy.head()

,abstract,title,author,affiliation_y,country,fuzzy_matched_QS,Abstract_Cleaned
0,The COVID-19 pandemic is impacting mental heal...,Natural Language Processing Reveals Vulnerable...,Laurie Rumker | Tanya Talkar | Daniel M ...,Department of Brain and Cognitive Sciences Mas...,United States | United States | United States ...,Massachusetts Institute of Technology (MIT) | ...,"[The, COVID, 19, pandemic, impacting, mental, ..."
1,"Brain regions in the ""pain matrix"", can be act...",Empathic control through coordinated interacti...,Nir Jacoby | Rebecca Saxe | Emile G Brun...,Department of Brain and Cognitive Sciences Mas...,United States | United States | United States ...,Massachusetts Institute of Technology (MIT) | ...,"[Brain, region, pain, matrix, activated, obser..."
2,During embryonic development of bilaterally sy...,Characterizing the diverse cells that associat...,N PenkoffLidbeck | C Schneider | M S Sch...,Computational and Systems Biology Massachusett...,United States | United States | Canada | Unite...,Massachusetts Institute of Technology (MIT) | ...,"[During, embryonic, development, bilaterally, ..."
3,Zebra finches (Taeniopygia guttata) are labora...,"Campylobacter taeniopygiae sp. nov., Campyloba...",Erin Bryant | Zeli Shen | Anthony Mannio...,Division of Comparative Medicine Massachusetts...,United States | United States | United States ...,Massachusetts Institute of Technology (MIT) | ...,"[Zebra, finch, Taeniopygia, guttata, laborator..."
4,Non-recurrent deep convolutional neural networ...,Evidence that recurrent circuits are critical ...,Kohitij Kar | Kailyn Schmidt | James J D...,McGovern Institute for Brain Research and Depa...,United States | United States | United States ...,Massachusetts Institute of Technology (MIT) | ...,"[Non, recurrent, deep, convolutional, neural, ..."


In [31]:
def display_topics(H, W, feature_names, title_list, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print('\n',"Topic %d:" % (topic_idx))
        print("Top Words: "," ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(title_list[doc_index])
            
no_top_words = 15
no_top_documents = 4     
title_list = copy['title'].tolist()
tf_feature_names = count_vectorizer.get_feature_names()
#display_topics(lda_H, lda_W, tf_feature_names, title_list, no_top_words, no_top_documents)

In [32]:
from sklearn.decomposition import NMF

nmf_model = NMF(n_components=9, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf_abstracts)
nmf_W = nmf_model.transform(tfidf_abstracts)
nmf_H = nmf_model.components_


 Topic 0:
Top Words:  the of and in to for that with is we as on was were this
Social cognition as a mediator variable between neurocognition and functional outcome in schizophrenia: empirical review and new results by structural equation modeling. 
Psychopathy-related traits and the use of reward and social information: a computational approach. 
Quality of life and menopause: Developing a theoretical model based on meaning in life, self-efficacy beliefs, and body image. 
Modeling an auditory stimulated brain under altered states of consciousness using the generalized Ising model. 

 Topic 1:
Top Words:  self efficacy health social care support and esteem perceived stigma diabetes adherence mental behaviors patients
Relationships between health literacy and heart failure knowledge, self-efficacy, and self-care adherence. 
Pathways of empowerment perceptions, health literacy, self-efficacy, and self-care behaviors to glycemic control in patients with type 2 diabetes mellitus. 
Identif